In [1]:
pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import AlbertTokenizerFast, AlbertForSequenceClassification, TrainingArguments, Trainer, EvalPrediction
from datasets import load_dataset
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from google.colab import drive

In [3]:
drive.mount('/content/drive')
dataset = load_dataset('csv', data_files={
                                          'train': '/content/drive/MyDrive/Colab Notebooks/df_balanceado_cinco_despachos.csv',
                                          'validation': '/content/drive/MyDrive/Colab Notebooks/df_validacao_small.csv',
                                          'test': '/content/drive/MyDrive/Colab Notebooks/df_teste_small.csv'
                                          })

Mounted at /content/drive


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
tokenizer = AlbertTokenizerFast.from_pretrained('albert-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

In [5]:
# Label map para o treinamento (6 classes)
label_map_train = {
    "acordao_de_2_instancia": 0,
    "agravo_em_recurso_extraordinario": 1,
    "outros": 2,
    "peticao_do_RE": 3,
    "sentenca": 4,
    "despacho_de_admissibilidade": 5
}

# Label map para a validação (6 classes)
label_map_validation = {
    "acordao_de_2_instancia": 0,
    "agravo_em_recurso_extraordinario": 1,
    "outros": 2,
    "peticao_do_RE": 3,
    "sentenca": 4,
    "despacho_de_admissibilidade": 5
}

# Label map para o teste (6 classes, incluindo a classe não vista)
label_map_test = {
    "acordao_de_2_instancia": 0,
    "agravo_em_recurso_extraordinario": 1,
    "outros": 2,
    "peticao_do_RE": 3,
    "sentenca": 4,
    "despacho_de_admissibilidade": 5
}

def preprocess_training_data(examples):
    examples["label"] = [label_map_train[label] for label in examples["document_type"]]
    return tokenizer(examples["body"], padding="max_length", truncation=True)

def preprocess_validation_data(examples):
    examples["label"] = [label_map_validation[label] for label in examples["document_type"]]
    return tokenizer(examples["body"], padding="max_length", truncation=True)

def preprocess_testing_data(examples):
    examples["label"] = [label_map_test[label] for label in examples["document_type"]]
    return tokenizer(examples["body"], padding="max_length", truncation=True)

encoded_dataset = {}

# Pré-processando o conjunto de treinamento
encoded_dataset["train"] = dataset["train"].map(preprocess_training_data, batched=True)

# Pré-processando o conjunto de validação
encoded_dataset["validation"] = dataset["validation"].map(preprocess_validation_data, batched=True)

# Pré-processando o conjunto de teste
encoded_dataset["test"] = dataset["test"].map(preprocess_testing_data, batched=True)

#encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/17742 [00:00<?, ? examples/s]

Map:   0%|          | 0/12631 [00:00<?, ? examples/s]

Map:   0%|          | 0/12118 [00:00<?, ? examples/s]

In [6]:
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=6)

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none",
)

In [8]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average="weighted", zero_division=0)
    recall = recall_score(labels, preds, average="weighted", zero_division=0)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [9]:
# Criar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    compute_metrics=compute_metrics
)

# Treinar o modelo
trainer.train()

# Avaliar o modelo
eval_results = trainer.evaluate()
print(f"Accuracy: {eval_results['eval_accuracy']}")
print(f"F1 Score: {eval_results['eval_f1']}")
print(f"Precision: {eval_results['eval_precision']}")
print(f"Recall: {eval_results['eval_recall']}")

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.783900,0.849606,0.703745,0.693898,0.697782,0.703745


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.783900,0.849606,0.703745,0.693898,0.697782,0.703745
2,0.610600,0.709392,0.755285,0.737969,0.743611,0.755285
3,0.497600,0.649554,0.777927,0.766601,0.760768,0.777927


Accuracy: 0.7779273216689099
F1 Score: 0.7666013226392111
Precision: 0.760767831473574
Recall: 0.7779273216689099
